In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'): 
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import wandb
from wandb.keras import WandbCallback
import os
#os.system('jupyter labextension update --all') #conda update jupyterlab jupyter labextension update --all
#os.system('jupyter labextension list')
os.system('! wandb login be213aaff4ff14945d480abc18697d8664bba8c8')
wandb.init(project='KaggleAugust2022', name='August2022')

In [ ]:
training = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/test.csv')
test['failure'] = np.NaN
training['train_test'] = 1
test['train_test'] = 0
all_data = pd.concat([training,test]) 
#all_data = all_data.dropna()
#all_data = all_data.fillna(np.NaN)
#all_data = all_data.drop(['product_code'], axis=1)
#all_data = all_data.drop(['id'], axis=1)

In [ ]:
test

In [ ]:
all_data

In [ ]:
#one-hot encoding
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
attribute_names = all_data.columns.tolist()
test_atts = test.columns.tolist()
print(attribute_names)
print('----------------------')
print(test_atts)
print(attribute_names == test_atts)

In [ ]:
all_dummies = pd.get_dummies(all_data[attribute_names])
test_dummies = pd.get_dummies(test[test_atts])  
X_train = all_dummies[all_data.train_test == 1].drop(['train_test'], axis=1)
for column in X_train.columns:
    mean_value = X_train[column].mean()
    X_train[column].fillna(mean_value, inplace=True)
X_check = X_train #for heatmap
X_train = X_train.drop(['failure'], axis=1)
X_test = all_dummies[all_data.train_test == 0].drop(['train_test'], axis=1)
y_train = all_data[all_data.train_test==1].failure
X_test = X_test.drop(['failure'], axis=1)
for column in X_test.columns:
    mean_value = X_test[column].mean()
    X_test[column].fillna(mean_value, inplace=True)

'''attr = ohe.fit_transform(all_data.columns).toarray()
X_train = all_dummies[all_data.train_test == 1].drop(['train_test'], axis=1)
X_test = all_dummies[all_data.train_test == 0].drop(['train_test'], axis=1)'''

def drop_zero_columns(df):
    zero_columns = df.columns[(df == 0).all()]
    df.drop(columns=zero_columns, inplace=True)

#drop_zero_columns(X_train)
#drop_zero_columns(X_test)
#y_train = all_data[all_data.train_test==1].dropna(axis=0).failure
#print(X_test.columns.tolist() == X_train.columns.tolist())

In [ ]:
X_train

In [ ]:
test

In [ ]:
training

In [ ]:
X_test

In [ ]:
print(X_test.isna().sum())

In [ ]:
X_atts = X_train.drop(['id', 'loading', #'failure', 
                       'product_code_A',
                       'product_code_B', 'product_code_C', 'product_code_D', 
                       'product_code_E', 
                       'attribute_0_material_5', 'attribute_0_material_7', 
                                      'attribute_1_material_5', 'attribute_1_material_6', 'attribute_1_material_8'],axis=1)
all_attributes = X_atts.columns.tolist()

In [ ]:
#feature engineering
#since the correlation of the data is very low almost everywhere and the names are unspecific,
#we cant really drop anything so I will just do some basic normalization

from sklearn.preprocessing import StandardScaler, MinMaxScaler
'''columns_to_normalize = ['measurement_0', 'measurement_1', 'measurement_2', 
                        'measurement_3', 'measurement_4', 'measurement_5', 
                        'measurement_6', 'measurement_7', 'measurement_8', 
                        'measurement_9', 'measurement_10', 'measurement_11', 
                        'measurement_12', 'measurement_13', 'measurement_14', 
                        'measurement_15', 'measurement_16', 'measurement_17']'''
columns_to_normalize = all_attributes
scaler = MinMaxScaler()
X_train_scaled = X_train
X_train_scaled[columns_to_normalize] = scaler.fit_transform(X_train[columns_to_normalize])
#X_train_scaled = X_train_scaled[:,~np.all(np.isnan(d), axis=0)]
X_test_scaled = X_test
X_test_scaled[columns_to_normalize] = scaler.fit_transform(X_test[columns_to_normalize])
#X_test_scaled = X_test_scaled.dropna()

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
y_train.sum()

In [ ]:
X_test_scaled

In [ ]:
X_test

In [ ]:
plt.figure(figsize=(20, 16))
#check_A = X_check.drop(['product_code_B', 'product_code_C', 'product_code_D', 'product_code_E'], axis=1)
sns.heatmap(X_check.corr(), annot=False, cmap='coolwarm')

plt.title('Correlation Heatmap')
plt.show()

In [ ]:
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
#from sklearn.model_selection import KFold
from sklearn import tree

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.ensemble import VotingClassifier



In [ ]:
import tensorflow as tf

def seed_everything():
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
    np.random.seed(hash("improves reproducibility") % 2**32 - 1)
    tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)
    
seed_everything()

In [ ]:
dt = tree.DecisionTreeClassifier(random_state = 1)
cv = cross_val_score(dt,X_train_scaled,y_train,cv=5, error_score='raise')
dt.fit(X_train,y_train)
print(cv)
print(cv.mean())
'''plt.figure(figsize=(12, 8))  # Adjust the figure size as needed
tree.plot_tree(dt, filled=True, feature_names=X_train.columns, class_names=True)  # Assuming X_train is a DataFrame

# Display the plot
plt.show()'''
'''num_cols_per_row = 3

# Calculate the number of rows needed for the subplots
num_features = X_train.shape[1]
num_rows = (num_features + num_cols_per_row - 1) // num_cols_per_row

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols_per_row, figsize=(15, 5*num_rows))

# Flatten the axes array if there's only one row of subplots
if num_rows == 1:
    axes = axes.reshape(1, -1)

# Loop through each feature and create a distplot
for i, feature in enumerate(X_train.columns):
    row = i // num_cols_per_row
    col = i % num_cols_per_row
    ax = axes[row, col]
    fig = sns.histplot(X_train[feature], ax=ax)
    ax.set_title(f'Distribution of {feature}')
    wandb.log({feature + 'distribution': fig})
    
# Remove any empty subplots
for i in range(num_features, num_rows * num_cols_per_row):
    fig.delaxes(axes.flatten()[i])
    

# Adjust spacing between subplots
fig.tight_layout()
plt.show()
'''
# Show the plot

#for feature in X_train.columns:
    #fig = sns.histplot(X_train[feature])
    #wandb.log({feature + "distribution": fig})
    #plt.show()


In [ ]:
print(X_train_scaled.isna().sum())
print(y_train.isna().sum())
print(X_test_scaled.isna().sum())

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


gnb = GaussianNB()
cv = cross_val_score(gnb,X_train_scaled,y_train,cv=5)
print('gnb')
print(cv)
print(cv.mean())
print('#######################')
lr = LogisticRegression(max_iter = 2000)
cv = cross_val_score(lr,X_train,y_train,cv=5)
print('lr not scaled')
print(cv)
print(cv.mean())
print('#######################')
lr = LogisticRegression(max_iter = 2000)
cv = cross_val_score(lr,X_train_scaled,y_train,cv=5)
print('lr scaled')
print(cv)
print(cv.mean())
print('#######################')
dt = tree.DecisionTreeClassifier(random_state = 1)
cv = cross_val_score(dt,X_train,y_train,cv=5)
print('dt not scaled')
print(cv)
print(cv.mean())
print('#######################')
dt = tree.DecisionTreeClassifier(random_state = 1)
cv = cross_val_score(dt,X_train_scaled,y_train,cv=5)
print('dt scaled')
print(cv)
print(cv.mean())
print('#######################')
knn = KNeighborsClassifier()
cv = cross_val_score(knn,X_train,y_train,cv=5)
print('knn not scaled')
print(cv)
print(cv.mean())
print('#######################')
knn = KNeighborsClassifier()
cv = cross_val_score(knn,X_train_scaled,y_train,cv=5)
print('knn scaled')
print(cv)
print(cv.mean())
print('#######################')
rf = RandomForestClassifier(random_state = 1)
cv = cross_val_score(rf,X_train,y_train,cv=5)
print('rf not scaled')
print(cv)
print(cv.mean())
print('#######################')
rf = RandomForestClassifier(random_state = 1)
cv = cross_val_score(rf,X_train_scaled,y_train,cv=5)
print('rf scaled')
print(cv)
print(cv.mean())
print('#######################')
svc = SVC(probability = True)
cv = cross_val_score(svc,X_train_scaled,y_train,cv=5)
print('svc')
print(cv)
print(cv.mean())
print('#######################')
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state =1)
cv = cross_val_score(xgb,X_train_scaled,y_train,cv=5)
print('xgboost')
print(cv)
print(cv.mean())
print('#######################')
#Voting classifier takes all of the inputs and averages the results. For a "hard" voting classifier each classifier gets 1 vote "yes" or "no" and the result is just a popular vote. For this, you generally want odd numbers
#A "soft" classifier averages the confidence of each of the models. If a the average confidence is > 50% that it is a 1 it will be counted as such
from sklearn.ensemble import VotingClassifier
#voting_clf = VotingClassifier(estimators = [('lr',lr),('knn',knn),('rf',rf),('gnb',gnb),('svc',svc),('xgb',xgb)], voting = 'soft') 
voting_clf = VotingClassifier(estimators = [('dt',dt),('gnb',gnb),('xgb',xgb)], voting = 'soft') 

In [ ]:
print("voting classifier:")
'''cv = cross_val_score(voting_clf,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())'''
#voting_clf.fit(X_train_scaled,y_train)


In [ ]:
#other attempts for classifiers
print('####1####')
voting_clf_1 = VotingClassifier(estimators = [('gnb',gnb),('lr',lr), ('svc',svc)], voting = 'soft')
cv_1 = cross_val_score(voting_clf_1,X_train_scaled,y_train,cv=5)
print(cv_1)
print(cv_1.mean())

'''print('####2####')
voting_clf_2 = VotingClassifier(estimators = [('dt',dt),('xgb',xgb)], voting = 'soft')
cv_2 = cross_val_score(voting_clf_2,X_train_scaled,y_train,cv=5)
print(cv_2)
print(cv_2.mean())

print('####3####')
voting_clf_3 = VotingClassifier(estimators = [('dt',dt),('gnb',gnb)], voting = 'soft')
cv_3 = cross_val_score(voting_clf_3,X_train_scaled,y_train,cv=5)
print(cv_3)
print(cv_3.mean())

print('####4####')
voting_clf_4 = VotingClassifier(estimators = [('lr',lr), ('dt',dt),('gnb',gnb),('xgb',xgb)], voting = 'soft')
cv_4 = cross_val_score(voting_clf_4,X_train_scaled,y_train,cv=5)
print(cv_4)
print(cv_4.mean())

print('####5####')
voting_clf_5 = VotingClassifier(estimators = [('lr',lr), ('dt',dt),('gnb',gnb),('xgb',xgb)], voting = 'hard')
cv_5 = cross_val_score(voting_clf_5,X_train_scaled,y_train,cv=5)
print(cv_5)
print(cv_5.mean())'''

####1####
[0.74614227 0.78678961 0.78735416 0.78641325 0.78735416]
0.7788106887467068
####2####
[0.21245766 0.74651863 0.65280391 0.23447497 0.71471584]
0.5121942039894618
####3####
[0.21245766 0.74651863 0.65280391 0.23447497 0.71471584]
0.5121942039894618
####4####
[0.21245766 0.78227324 0.78039142 0.24124953 0.78735416]
0.5607452013549116
####5####
[0.25818592 0.78660143 0.78716598 0.76759503 0.78735416]
0.6773805043281896

In [ ]:
'''print('####6####')
voting_clf_6 = VotingClassifier(estimators = [('gnb',gnb),('lr',lr), ('svc',svc),('xgb',xgb)], voting = 'soft')
cv_6 = cross_val_score(voting_clf_6,X_train_scaled,y_train,cv=5)
print(cv_6)
print(cv_6.mean())'''

In [ ]:
X_train_scaled

In [ ]:
X_test_scaled

In [ ]:
voting_clf.fit(X_train_scaled,y_train)

In [ ]:
voting_clf_1.fit(X_train_scaled,y_train)

In [ ]:
voting_clf_2 = VotingClassifier(estimators = [('dt',dt), ('rf',rf)], voting = 'soft')

In [ ]:
'''feat_importances = pd.Series(rf.feaure_importances_, index=X_train_scaled.columns)
feat_importances.nlargest(20).plot(kind='barh')'''

In [ ]:
y_hat_base_vc = voting_clf_1.predict(X_test).astype(float)
#y_hat_base_vc = voting_clf.predict(X_train).astype(float)
dt.fit(X_train_scaled,y_train)
y_hat_df = dt.predict(X_test).astype(float)
voting_clf_2.fit(X_train_scaled,y_train)
y_hat_3 = voting_clf_2.predict(X_test).astype(float)
submission = {'id': test.id , 'failure': y_hat_base_vc}
submission = pd.DataFrame(data=submission)
submission.to_csv('submission.csv', index=False)

In [ ]:
'''import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay
ax = plt.gca()
rfc_disp = RocCurveDisplay.from_estimator(voting_clf, X_test, y_hat_base_vc, ax=ax, alpha=0.8)
#svc_disp.plot(ax=ax, alpha=0.8)'''

In [ ]:
print(y_hat_base_vc)
print(y_hat_base_vc.sum())
print(y_hat_3)
print(y_hat_3.sum())
print(y_hat_df)
print(y_hat_df.sum())

In [ ]:
#os.system('pip install --upgrade matplotlib')

In [ ]:
# Create a Matplotlib figure explicitly
fig, ax = plt.subplots(figsize=(8, 6))

# Plot learning curve on the specified figure
def plot_learning_curve(estimator, X, y, cv=None, train_sizes=np.linspace(0.1, 1.0, 5)):
    # ... (rest of the code)
    ax.fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    ax.fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    ax.plot(train_sizes, train_scores_mean, "o-", color="r", label="Training score")
    ax.plot(train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score")
    ax.legend(loc="best")

# Log the Matplotlib figure to W&B
wandb.log({"learning_curve": fig})

In [ ]:
def plot_learning_curve(estimator, X, y, cv=None, train_sizes=np.linspace(0.1, 1.0, 5)):
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, train_sizes=train_sizes, scoring="accuracy", n_jobs=-1
    )

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.figure(figsize=(8, 6))
    plt.title("Learning Curve")
    plt.xlabel("Training examples")
    plt.ylabel("Score")

    plt.fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    plt.fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    plt.plot(train_sizes, train_scores_mean, "o-", color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score")
    plt.legend(loc="best")

    # Save the plot as an image file
    plt.savefig("learning_curve.png")

# Assuming you have training data X_train and labels y_train
# You can customize the cross-validation strategy by specifying the 'cv' parameter
plot_learning_curve(voting_clf, X_train, y_train)

# Log the saved image to W&B
wandb.log({"learning_curve": wandb.Image("learning_curve.png")})

# Finish the run
wandb.finish()

In [ ]:
def plot_learning_curve(estimator, X, y, cv=None, train_sizes=np.linspace(0.1, 1.0, 5)):
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, train_sizes=train_sizes, scoring="accuracy", n_jobs=-1
    )

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.figure(figsize=(8, 6))
    plt.title("Learning Curve")
    plt.xlabel("Training examples")
    plt.ylabel("Score")

    plt.fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    plt.fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    plt.plot(train_sizes, train_scores_mean, "o-", color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score")
    plt.legend(loc="best")

    # Log the plot to W&B
    wandb.log({"learning_curve": plt})

# Assuming you have training data X_train and labels y_train
# You can customize the cross-validation strategy by specifying the 'cv' parameter
plot_learning_curve(voting_clf, X_train, y_train) #.columns.tolist()

# Finish the run
wandb.finish()